# TMA Analysis code supporting technote SITCOMTN-067
Craig Lage - 15-Nov-23 

This notebook characterizes TMA velocity, acceleration, and jerk. 


## Prepare the notebook

In [ ]:
# Directory to store the data
from pathlib import Path
data_dir = Path("./plots")
data_dir.mkdir(exist_ok=True, parents=True)
dayObs = 20231129

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from astropy.time import Time, TimeDelta
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker

# First, let's get the events

In [ ]:
client = EfdClient("idf_efd")
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(dayObs)
Nslews = 0
Ntracks = 0
for event in events:
    if event.type.name == 'TRACKING':
        Ntracks += 1
    elif event.type.name == 'SLEWING':
        Nslews += 1
print(f"There are {len(events)} events, {Nslews} slewing and {Ntracks} tracking")    

In [ ]:
events[161]

In [ ]:
events[125]

# Now plot the Velocity/Accel/Jerk plots.

In [ ]:
# Plotting the Vel/Accel/Jerk plot
%matplotlib inline
maxAzVels = []
maxAzAccels = []
maxAzJerks = []
maxElVels = []
maxElAccels = []
maxElJerks = []
smoothingFactor = 0.2 # In spline creation
kernel_size = 100 # In convolution
kernel = np.ones(kernel_size) / kernel_size

fig = plt.figure(figsize = (8,4))
for event in [events[92]]:
    try:
        if event.type.name != 'SLEWING':
            # Only slewing events
            continue
        start = event.begin - TimeDelta(2.0, format='sec')
        end = event.end + TimeDelta(2.0, format='sec')
        plotAz = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                                    ['*'],  start, end)
        plotEl = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                                    ['*'],  start, end)    
        azPos = await client.select_time_series('lsst.sal.MTMount.logevent_azimuthInPosition', \
                                                    ['inPosition', 'private_kafkaStamp'],  start, end)
        azPos = azPos[azPos['inPosition']] # Select only the True values
        elPos = await client.select_time_series('lsst.sal.MTMount.logevent_elevationInPosition', \
                                                    ['inPosition', 'private_kafkaStamp'],  start, end)
        elPos = elPos[elPos['inPosition']] # Select only the True values
        
        # Now calculates the spline fit and differentiate it to get the acceleration and jerk
        azPs = plotAz['actualPosition'].values
        azVs = plotAz['actualVelocity'].values
        azXs = plotAz['timestamp'].values - plotAz['timestamp'].values[0]  
        elPs = plotEl['actualPosition'].values
        elVs = plotEl['actualVelocity'].values
        elXs = plotEl['timestamp'].values - plotEl['timestamp'].values[0]
        plotStart = azXs[0] + 1.0
        plotEnd = azXs[-1] - 1.0

        
        start_slew = event.begin.unix_tai - plotAz['timestamp'].values[0]
        end_slew = event.end.unix_tai - plotAz['timestamp'].values[0]
        az_inPos = azPos['private_kafkaStamp'].values[0] - plotAz['timestamp'].values[0]  
        el_inPos = elPos['private_kafkaStamp'].values[0] - plotEl['timestamp'].values[0] 
        inPos = max(az_inPos, el_inPos)
        print(ss_time, ip_time, inPos)

        deltaAz = abs(azPs[-1] - azPs[0])  
        deltaEl = abs(elPs[-1] - elPs[0]) 
        if deltaAz < 1.0 and deltaEl < 1.0:
            # eliminate very small slews
            continue
        
        plotAzXs = np.linspace(azXs[0], azXs[-1], 2000)
        plotElXs = np.linspace(elXs[0], elXs[-1], 2000)
        azPSpline = UnivariateSpline(azXs, azPs, s=0)
        azVelSpline1 = UnivariateSpline(azXs, azVs, s=0) 
        #azVelSpline1 =azPSpline.derivative(n=1)
        # Now smooth the derivative before differentiating again
        smoothedAzVel = np.convolve(azVelSpline1(plotAzXs), kernel, mode='same')
        azVelSpline = UnivariateSpline(plotAzXs, smoothedAzVel, s=smoothingFactor)
        azAccSpline1 = azVelSpline.derivative(n=1)
        smoothedAzAcc = np.convolve(azAccSpline1(plotAzXs), kernel, mode='same')
        # Now smooth the derivative before differentiating again
        azAccSpline = UnivariateSpline(plotAzXs, smoothedAzAcc, s=smoothingFactor)
        azJerkSpline = azAccSpline.derivative(n=1) 
        elPSpline = UnivariateSpline(elXs, elPs, s=0)
        elVelSpline1 = UnivariateSpline(elXs, elVs, s=0)
        #elVelSpline1 =elPSpline.derivative(n=1)
        # Now smooth the derivative before differentiating again
        smoothedElVel = np.convolve(elVelSpline1(plotElXs), kernel, mode='same')
        elVelSpline = UnivariateSpline(plotElXs, smoothedElVel, s=smoothingFactor)
        elAccSpline1 = elVelSpline.derivative(n=1)
        smoothedElAcc = np.convolve(elAccSpline1(plotElXs), kernel, mode='same')
        # Now smooth the derivative before differentiating again
        elAccSpline = UnivariateSpline(plotElXs, smoothedElAcc, s=smoothingFactor)
        elJerkSpline = elAccSpline.derivative(n=1) 
    
        
        fig.clear()
        plt.subplots_adjust(wspace=0.3, hspace=0.5)
        plt.suptitle(f"MT Mount Slews - {dayObs} - {event.seqNum:03}", fontsize = 18)
        
        plt.subplot(1,2,1)
        plt.plot(plotAzXs, azPSpline(plotAzXs), lw=3, color='r', label='Spline fit')
        plt.scatter(azXs, azPs, marker='x', color='red', s=100, label='Measured points')
        plt.plot([start_slew, start_slew], [np.min(azPs), np.max(azPs)], ls='--', color='black', label='Slew Start')
        plt.plot([end_slew, end_slew], [np.min(azPs), np.max(azPs)], ls='-.', color='magenta', label='Slew End')
        plt.plot([inPos, inPos], [np.min(azPs), np.max(azPs)], ls='--', color='blue', label='In Position')
        plt.title(f"Azimuth")
        plt.ylabel("Degrees")
        plt.xlim(plotStart, plotEnd)
        plt.legend()
        
        plt.subplot(1,2,2)
        plt.plot(plotElXs, elPSpline(plotElXs), lw=3, color='g', label='Spline fit')
        plt.scatter(elXs, elPs, marker='x', color='g', s=100, label='Measured points')
        plt.plot([start_slew, start_slew], [np.min(elPs), np.max(elPs)], ls='--', color='black', label='Slew Start')
        plt.plot([end_slew, end_slew], [np.min(elPs), np.max(elPs)], ls='-.', color='magenta', label='Slew End')
        plt.plot([inPos, inPos], [np.min(elPs), np.max(elPs)], ls='--', color='blue', label='In Position')
        plt.title(f"Elevation")
        plt.ylabel("Degrees")
        plt.xlim(plotStart, plotEnd)
        plt.legend()
        plt.savefig(str(data_dir / f"Event_Debug_{dayObs}_{event.seqNum:03}.png"))
    except:
        continue


In [ ]:
eventMaker.printEventDetails(events[28])

In [ ]:
eventMaker.printEventDetails(events[125])

In [ ]:
events[92]

In [ ]:
eventMaker.printEventDetails(events[92])